In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 

In [6]:
df = pd.read_csv("amazon_cells_labelled.txt", delimiter="\t", header=None, names=["Review", "Sentiment"]) #delimeter,header, names used bcs of .txt file

In [ ]:
df.head() #first 5 rows

In [ ]:
df.shape

In [16]:
train_X, test_X, train_y, test_y = train_test_split(df["Review"], df["Sentiment"], test_size=0.2, random_state=42) #data split

In [ ]:
train_X.head()

In [ ]:
train_X.shape

In [ ]:
test_X.head()

In [ ]:
test_X.shape

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer #divide sequence of words into pieces
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X)

In [ ]:
tokenizer.word_index

In [25]:
train_sequences = tokenizer.texts_to_sequences(train_X)
test_sequences = tokenizer.texts_to_sequences(test_X)

In [ ]:
train_sequences

In [ ]:
max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
max_length

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
train_data_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [ ]:
train_data_padded

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [36]:
vocab_size = len(tokenizer.word_index) + 1 #
embedding_dim = 16
model = Sequential() #start of the model
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(8))
model.add(Dense(1, activation='sigmoid')) #binary output 0 or 1
#multiClass for 5 outputs
# model.add(Dense(5, activation='softmax')) 

In [ ]:
#compile the model architecture
optimizer = Adam(learning_rate=0.001) #use adam #change learning rate(0.01) //if want use 0.01 or 0.001
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) #if binary => binary_crossentropy ,if multiclass => categorical_crossentropy
history = model.fit(train_data_padded, train_y, epochs=10, batch_size=32, validation_split=0.1) #fit means training the dataset #validation_split(must)